In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt
def load_file(filename):
    cnt = BBCIDataset(filename).load()
    cnt = cnt.drop_channels(['STI 014'])
    def car(a):
        return a - np.mean(a, keepdims=True, axis=0)

    cnt = mne_apply(
        car, cnt)
    log.info("Resampling train...")
    cnt = resample_cnt(cnt, 250.0)
    return cnt


def create_set(cnt):
    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                             ('Rest', [3]), ('Feet', [4])])
    ival = [500,1520]
    log.info("Standardizing train...")
    cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                         cnt)
    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    return dataset

def create_inputs(dataset):
    x_right = dataset.X[dataset.y == 0]

    x_rest = dataset.X[dataset.y == 2]

    inputs_a = np_to_var(x_right[:,:,:,None], dtype=np.float32)

    inputs_b = np_to_var(x_rest[:,:,:,None], dtype=np.float32)
    inputs = [inputs_a, inputs_b]
    return inputs


In [ ]:
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = orig_train_cnt.reorder_channels(['C3', 'C4'])
train_cnt = resample_cnt(train_cnt, 125.0)
train_set = create_set(train_cnt)

train_inputs = create_inputs(train_set)

In [ ]:
orig_test_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/test/4.mat')
test_cnt = orig_test_cnt.reorder_channels(['C3', 'C4'])
test_cnt = resample_cnt(test_cnt, 125.0)
test_set = create_set(test_cnt)
test_inputs = create_inputs(test_set)

In [ ]:
cuda = False
if cuda:
    train_inputs  = [i.cuda() for i in train_inputs]
    test_inputs  = [i.cuda() for i in test_inputs]

In [ ]:
train_set.X.shape

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())

In [ ]:
trial = var_to_np(train_inputs[1])[0,0].squeeze()
ffted = np.fft.rfft(trial)
ffted[:5] = 0
ffted[15:] = 0
iffted = np.fft.irfft(ffted, n=128,axis=-1)
plt.figure(figsize=(16,4))
plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
plt.plot(iffted)
ffted = np.fft.rfft(trial)
ffted[5:] = 0
iffted = np.fft.irfft(ffted, n=128,axis=-1)
plt.plot(iffted)
#ffted = np.fft.rfft(trial)
#ffted[:15] = 0
#iffted = np.fft.irfft(ffted, n=256,axis=-1)
#plt.plot(iffted)

In [ ]:
coeffs = th.ones(31, requires_grad=True)
weights = th.ones(8, requires_grad=True)
optimizer = th.optim.Adam([coeffs, weights], lr=1e-2)

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    out = F.conv_transpose1d(coeffs.unsqueeze(0).unsqueeze(0), weights.unsqueeze(0).unsqueeze(0),
                      stride=4).squeeze()

    diff = out - train_inputs[1][0,0].squeeze()
    loss = th.mean(th.abs(diff))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:E}".format(loss.item()))
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
        plt.plot(var_to_np(out))
        display(fig)
        plt.close(fig)
        

In [ ]:
coeffs = th.ones(31, requires_grad=True)
weights = th.ones(8, requires_grad=True)
optimizer = th.optim.Adam([coeffs, weights], lr=1e-2)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    out = F.conv_transpose1d(coeffs.unsqueeze(0).unsqueeze(0), weights.unsqueeze(0).unsqueeze(0),
                      stride=4).squeeze()

    diff = out - train_inputs[1][0,0].squeeze()
    loss = th.mean(diff * diff)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:E}".format(loss.item()))
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
        plt.plot(var_to_np(out))
        display(fig)
        plt.close(fig)
        

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(var_to_np(weights))

In [ ]:
coeffs = th.ones(31, requires_grad=True)
weights = th.ones(8, requires_grad=True)
optimizer = th.optim.Adam([coeffs, ], lr=1e-2)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    out = F.conv_transpose1d(coeffs.unsqueeze(0).unsqueeze(0), weights.unsqueeze(0).unsqueeze(0),
                      stride=4).squeeze()

    diff = out - train_inputs[1][0,0].squeeze()
    loss = th.mean(diff * diff)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:E}".format(loss.item()))
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
        plt.plot(var_to_np(out))
        display(fig)
        plt.close(fig)
        

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(var_to_np(coeffs))

In [ ]:
coeffs = th.ones(15, requires_grad=True)
weights = th.ones(16, requires_grad=True)
optimizer = th.optim.Adam([coeffs, weights], lr=1e-2)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    out = F.conv_transpose1d(coeffs.unsqueeze(0).unsqueeze(0), weights.unsqueeze(0).unsqueeze(0),
                      stride=8).squeeze()

    diff = out - train_inputs[1][0,0].squeeze()
    loss = th.mean(diff * diff)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:E}".format(loss.item()))
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
        plt.plot(var_to_np(out))
        display(fig)
        plt.close(fig)
        

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(var_to_np(weights))

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(var_to_np(coeffs))

In [ ]:
coeffs_2d = var_to_np(coeffs[:14].view(-1,2))

plt.figure(figsize=(4,4))
plt.scatter(coeffs_2d[:,0], coeffs_2d[:,1], marker='o',
           color=cm.coolwarm(np.linspace(0,1,7)))
plt.plot(coeffs_2d[:,0], coeffs_2d[:,1], color='black',lw=0.5)
plt.xlim(-np.max(np.abs(plt.xlim())), np.max(np.abs(plt.xlim())))
plt.ylim(-np.max(np.abs(plt.ylim())), np.max(np.abs(plt.ylim())))

In [ ]:
128/7

In [ ]:
plt.plot(np.arctan2(coeffs_2d[:,1], coeffs_2d[:,0]))

In [ ]:
plt.plot((np.diff(np.arctan2(coeffs_2d[:,1], coeffs_2d[:,0]))) % (2*np.pi))

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(var_to_np(train_inputs[1])[0,0].squeeze())
plt.plot(np.repeat(np.mean(coeffs_2d ** 2, axis=1),18))
plt.plot(np.repeat(np.arctan2(coeffs_2d[:,1], coeffs_2d[:,0]),18))

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(np.fft.rfftfreq(256, d=1/250.0), np.abs(np.fft.rfft(var_to_np(train_inputs[1])[0,0].squeeze())))
plt.xticks(range(0,126,5));

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(np.fft.rfftfreq(256, d=1/250.0), np.mean(np.abs(np.fft.rfft(var_to_np(train_inputs[1])[:,0].squeeze())), axis=0))
plt.xticks(range(0,126,5));

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(np.fft.rfftfreq(256, d=1/250.0), np.abs(np.fft.rfft(train_set.X[:,0].squeeze())).T)
plt.xticks(range(0,126,5));